In [ ]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

TABELA_ORIGEM = "spotify_analytics.silver.tb_tracks"
TABELA_DESTINO = "spotify_analytics.gold.dm_track"

### Configuração

Define origem (Silver tb_tracks) e destino (Gold dm_track).

### Transformação Silver → Gold (Dimensão Track)

Lê tb_tracks (Silver) e cria dimensão com:
1. **Mapeamento de campos**: track_id → id_track_spotify
2. **Coluna derivada de negócio**: 
   - `ds_duracao` baseada em duration_ms:
     - < 180000ms (3min): "Curta"
     - 180000-300000ms (3-5min): "Média"
     - > 300000ms (5min): "Longa"

**Decisão Arquitetural**: Incluir informações do álbum na dimensão track:
- **Vantagem**: Queries simples sem JOIN adicional
- **Trade-off**: Duplicação de dados de álbum
- **Justificativa**: Não temos volume suficiente para justificar dm_album separada

Faz MERGE em dm_track (Gold).

In [ ]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("track_id").alias("id_track_spotify"),
    F.col("track_name").alias("nm_track"),
    F.col("album_name").alias("nm_album"),
    F.col("album_type").alias("ds_album_type"),
    
    # Coluna derivada de negócio
    F.when(F.col("duration_ms") < 180000, "Curta")
     .when(F.col("duration_ms") < 300000, "Média")
     .otherwise("Longa")
     .alias("ds_duracao")
)

print(f"Total de tracks: {df_gold.count()}")
df_gold.show(10, truncate=False)

### MERGE em dm_track

Match por id_track_spotify (natural key).

In [ ]:
DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"),
    "t.id_track_spotify = s.id_track_spotify"
).whenMatchedUpdate(
    set = {
        "nm_track": "s.nm_track",
        "nm_album": "s.nm_album",
        "ds_album_type": "s.ds_album_type",
        "ds_duracao": "s.ds_duracao"
    }
).whenNotMatchedInsert(
    values = {
        "id_track_spotify": "s.id_track_spotify",
        "nm_track": "s.nm_track",
        "nm_album": "s.nm_album",
        "ds_album_type": "s.ds_album_type",
        "ds_duracao": "s.ds_duracao"
    }
).execute()

print(f"✅ Dimensão Track carregada!")

### Verificação Final

In [ ]:
spark.sql(f"""
    SELECT ds_duracao, COUNT(*) as qtd_tracks
    FROM {TABELA_DESTINO}
    GROUP BY ds_duracao
    ORDER BY qtd_tracks DESC
""").show()

spark.sql(f"""
    SELECT ds_album_type, COUNT(*) as qtd_tracks
    FROM {TABELA_DESTINO}
    GROUP BY ds_album_type
    ORDER BY qtd_tracks DESC
""").show()